## Based on the PyTorch quickstart tutorial ##

#### https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html ####

#### Sections:
- [Working with Data](#working-with-data)
- [Creating Models](#creating)
- [Optimizing the Model Parameters](#optimizing)
- [Saving and Loading Models](#saving-and-loading-models)
####



<a id="working-data"></a>
### Working with Data ###

In [ ]:
#PyTorch primitives: torch.utils.data.Dataset stores samples and corresponding labels.
#torch.utils.data.DataLoader wraps an iterable around the Dataset to enable easy access to the samples.

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets #package consists of popular datasets, model architectures, and common image transformations for computer vision.
from torchvision.transforms import ToTensor

In [ ]:
#Using FashionMNIST dataset
#Dataset includes two arguments: transform and target_transform to modify the samples and labels respectively.
#Download training and test data from open datasets.

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
#Pass the dataset to a DataLoader: iterable, automatic batching, sampling, shuffling and multiprocess data loading.

#Create data loaders
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

#Iterate through the DataLoader
#The FashionMNIST dataset consists of 60,000 training samples and 10,000 testing samples.
for X, y in test_dataloader:
    #features and labels
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


<a id="creating"></a>
### Creating Models ###

In [ ]:
#Create a class that inherits from nn.Module.
#Define the layers of the model in the __init__ method.
#Specify how data flows through the network in the forward method.

#Accelerate using accelerator, if possible.
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

#Define the model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

#Instantiate the model
model = NeuralNetwork().to(device)
print(model)




Using mps device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


<a id="optimizing"></a>
### Optimizing the Model Parameters ###

In [ ]:
#Define loss function and optimizer
#Loss function: measures how well the model performed
#Optimizer: updates the model parameters based on the loss function

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

#Training loop
#Iterate through the dataset multiple times
#For each iteration, perform the following steps:
#1. Forward pass: Compute predicted outputs by passing inputs to the model.
#2. Compute loss: Calculate the loss using the predicted and true outputs.
#3. Backward pass: Compute gradients of the loss with respect to model parameters.
#4. Update weights: Adjust model parameters based on gradients.

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    print(f"size: {size}")
    model.train() #set the model to training mode
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        #Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        #Backpropagation
        loss.backward() #compute gradients of the loss with respect to model parameters
        optimizer.step() #update model parameters based on gradients
        optimizer.zero_grad() #zero the gradients before the backward pass
        
        #Print loss every 100 batches
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            

In [ ]:
def test(dataloader, model, loss_fn):
    size= len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() #set the model to evaluation mode
    test_loss, correct = 0, 0
    with torch.no_grad(): #no gradients are computed
        for X, y in dataloader:
            X, y = X.to(device), y.to(device) #move data to the device
            pred = model(X) #forward pass
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
#The training process is conducted over several iterations (epochs).

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
size: 60000
loss: 2.289942 [    0/60000]
loss: 2.291219 [ 6400/60000]
loss: 2.276417 [12800/60000]
loss: 2.279450 [19200/60000]
loss: 2.253651 [25600/60000]
loss: 2.226903 [32000/60000]
loss: 2.225408 [38400/60000]
loss: 2.188240 [44800/60000]
loss: 2.180995 [51200/60000]
loss: 2.162608 [57600/60000]
Test Error: 
 Accuracy: 43.0%, Avg loss: 2.153136 

Epoch 2
-------------------------------
size: 60000
loss: 2.148113 [    0/60000]
loss: 2.143168 [ 6400/60000]
loss: 2.091084 [12800/60000]
loss: 2.110883 [19200/60000]
loss: 2.044011 [25600/60000]
loss: 1.994117 [32000/60000]
loss: 2.016250 [38400/60000]
loss: 1.933272 [44800/60000]
loss: 1.932219 [51200/60000]
loss: 1.874278 [57600/60000]
Test Error: 
 Accuracy: 55.4%, Avg loss: 1.867145 

Epoch 3
-------------------------------
size: 60000
loss: 1.891934 [    0/60000]
loss: 1.861511 [ 6400/60000]
loss: 1.749193 [12800/60000]
loss: 1.789434 [19200/60000]
loss: 1.677787 [25600/60000]
loss: 1.636984 

<a id="saving-loading"></a>
### Saving and Loading Models ###

In [ ]:
#A common way to save a model is to serialize the internal state dictionary (containing the model parameters).
# The process for loading a model includes re-creating the model structure and loading the state dictionary into it.

torch.save(model.state_dict(), "model.pth") #save the model
print("Saved PyTorch Model State to model.pth")

#Load the model
model = NeuralNetwork().to(device) #recreate the model structure
model.load_state_dict(torch.load("model.pth", weights_only=True))  #load the state dictionary


Saved PyTorch Model State to model.pth


<All keys matched successfully>

In [ ]:
#The model can be used to make predictions on new data.

classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval() #set the model to evaluation mode
x, y = test_data[0][0], test_data[0][1] #get the first sample
with torch.no_grad():
    x = x.to(device) #move data to the device
    pred = model(x) #forward pass
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted: {predicted}, Actual: {actual}")

Predicted: Ankle boot, Actual: Ankle boot
